# weibo 크롤링

## 크롤링 조건 설정

In [105]:
# 페이지 스크롤 다운 횟수 설정(숫자가 클수록 많은 페이지 크롤링)
num_scrolldown = 4
# 다음 페이지 클릭 횟수
page_number = 1
# 날짜 설정 : "설정 날짜" 포함, 이후 날짜 데이터 출력됨.
start_date = '2022-01-01'

In [2]:
from bs4 import BeautifulSoup as bs
import time
import random
from selenium import webdriver
import pandas as pd
from datetime import datetime
import getpass
import chromedriver_autoinstaller as ca

### 페이지 로그인

In [91]:
# 사이트 로그인
ca.install(True)
driver = webdriver.Chrome()
driver.get("https://weibo.com/login.php")
time.sleep(5)
driver.implicitly_wait(15)

# 창 최대화
driver.maximize_window()
time.sleep(1)

In [92]:
# 로그인 정보
id = input("ID를 입력하세요 : ")
password = getpass.getpass("password(비밀번호)를 입력하세요 : ")

element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
time.sleep(1)
element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')

element_id.clear()
element_password.clear()

element_id.send_keys(id)
time.sleep( random.uniform(0.5, 1) )
element_password.send_keys(password)
time.sleep( random.uniform(0.5, 1) )

##로그인버튼 클릭
driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()
time.sleep(5)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_19156/3677500735.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
C:\Users\kbjung\AppData\Local\Temp/ipykernel_19156/3677500735.py:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
C:\Users\kbjung\AppData\Local\Temp/ipykernel_19156/3677500735.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()


### 크롤링할 페이지 접속

In [97]:
# 페이지 접속
url = 'https://weibo.com/p/1001066011831795/home?from=page_100106&mod=TAB#place'
# webdriver 로드
driver.get(url)
time.sleep(5)
driver.implicitly_wait(5)

## 데이터 수집

In [98]:
content_list = []
number_view_list = []
number_like_list = []
number_share_list = []
number_comment_list = []
date_list = []
number_follower_list = []
link_list = []

for p in range(page_number + 1):
    # 페이지 스크롤 다운
    for i in range(num_scrolldown):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep( random.uniform(1,3) )
        driver.implicitly_wait(5)
    
    # 페이지 크롤링
    soup = bs(driver.page_source, 'lxml')
    
    # 타이틀 수집
    # 타이틀 첫번째 마침표까지만 가져오기
    raw_contents = soup.find_all('div', class_='WB_text W_f14')
    num = len(raw_contents)
    for i in range(num):
        content = raw_contents[i].text.strip().split('。')[0]
        content_list.append(content)
    #
    #  조회수 수집
    raw_number_view = soup.find_all('i', class_='S_txt2')
    for i in range(len(raw_number_view)):
        temp = raw_number_view[i].attrs['title']
        number_view = int(temp[9:-1])
        number_view_list.append(number_view)
    
    # 정보 수집
    raw_info = soup.find_all('span', 'line S_line1')
    
    # 좋아요 수 수집
    num_range = (len(raw_info) - 3) // 4 + 1
    raw_info = soup.find_all('span', 'line S_line1')
    for i in range(num_range):
        num = ( i + 1 ) * 4 - 1
        temp = raw_info[num]
        temp = temp.find_all('em')[-1].text
        if temp == '赞':
            number_like = 0
        else:
            number_like = int(temp)
        number_like_list.append(number_like)
    
    # 공유 수 수집
    num_range = (len(raw_info) - 3) // 4 + 1
    raw_info = soup.find_all('span', 'line S_line1')
    for i in range(num_range):
        num = ( i + 1 ) * 4 - 3
        temp = raw_info[num]
        temp = temp.find_all('em')[-1].text
        if '转发' in temp:
            number_share_list.append(0)
        else:
            number_like = int(temp)
            number_share_list.append(number_like)

    # 댓글수 수집
    num_range_like = (len(raw_info) - 3) // 4 + 1
    raw_info = soup.find_all('span', 'line S_line1')
    for i in range(num_range_like):
        num = ( i + 1 ) * 4 - 2
        temp = raw_info[num]
        temp = temp.find_all('em')[-1].text
        if '评论' in temp:
            number_comment_list.append(0)
        else:
            number_like = int(temp)
            number_comment_list.append(number_like)
    
    # 게시글 날짜 수집
    raw_date = soup.find_all('div', 'WB_from S_txt2')
    for i in range(len(raw_date)):
        temp = raw_date[i].find('a', 'S_txt2')
        date = temp.attrs['title'][:10]
        date_list.append(date)

    # 팔로워 수 수집
    number_of_follower = soup.find_all('strong', 'W_f16')
    num = int(number_of_follower[1].text)
    number_follower_list.append(num)
    for i in range(len(raw_contents)-1):
        number_follower_list.append(0)
    
    # 링크 수집
    raw_links = soup.find_all('div', 'WB_from S_txt2')
    for link in raw_links:
        temp = 'https://weibo.com' + link.find('a' ,'S_txt2').attrs['href']
        link_list.append(temp)

    # 다음 페이지 버튼 클릭
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[2]/div[2]/div[4]/div/div[47]/div/a').click()
    time.sleep( random.uniform(1,3) )
    driver.implicitly_wait(5)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_19156/1438168588.py:98: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/div[2]/div[2]/div[4]/div/div[47]/div/a').click()


### 크롤링 데이터 확인

In [99]:
print('게시글 수 : ', len(content_list))
print('조회수 리스트 원소수 : ', len(number_view_list))
print('좋아요 리스트 원소수 : ', len(number_like_list))
print('공유수 리스트 원소수 : ', len(number_share_list))
print('댓글수 리스트 원소수 : ', len(number_comment_list))
print('날짜 리스트 원소수 : ', len(date_list))
print('팔로워 수 리스트 원소수 : ', len(number_follower_list))
print("링크 리스트 원소수 : ", len(link_list))

게시글 수 :  90
조회수 리스트 원소수 :  90
좋아요 리스트 원소수 :  90
공유수 리스트 원소수 :  90
댓글수 리스트 원소수 :  90
날짜 리스트 원소수 :  90
팔로워 수 리스트 원소수 :  90
링크 리스트 원소수 :  90


In [100]:
# chromedriver 종료
driver.quit()

### 데이터 셋 생성

In [101]:
# 데이터 프레임 생성
df = {'날짜':date_list, '제목':content_list, '조회수':number_view_list, '좋아요_수':number_like_list, '공유수':number_share_list, '댓글수':number_comment_list, '팔로워_수':number_follower_list, '링크':link_list}
data = pd.DataFrame(df)
data.head()

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
0,2022-05-20,"京畿道将举办""全家人一起参与的游戏秀""2022Play博览会京畿道是韩国游戏产业的圣地",1367,0,0,0,544332,https://weibo.com/6011831795/LtSxwgnLU?from=pa...
1,2022-05-18,京畿道杨平吉兆酒店在京畿道可以见到一个日式建筑物，具有异国风情的地方,6441,2,0,0,0,https://weibo.com/6011831795/LtAldcubF?from=pa...
2,2022-05-16,京畿道杨平大型户外咖啡店 JUN179京畿道杨平拥有广阔的空间并衔接着龙门川溪谷，是感受大自...,6199,3,2,0,0,https://weibo.com/6011831795/LthBqiDCm?from=pa...
3,2022-05-13,京畿道，举办时隔3年回归的线下游戏秀”PlayX4”📍 指南 : O网页链接#京畿道##京畿...,6723,4,0,0,0,https://weibo.com/6011831795/LsPpKv8Gi?from=pa...
4,2022-05-11,道，“创意科学培养体验中心” 开馆…培养无人机、宇宙产业等未来人才在京畿道坡州和杨平未来教育...,7074,3,0,0,0,https://weibo.com/6011831795/LswxZh38m?from=pa...


### 데이터 셋. 날짜로 슬라이싱

In [102]:
# 데이터 셋 날짜로 슬라이싱
slicing_data = data[data['날짜'] >= start_date]
slicing_data.head()

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
0,2022-05-20,"京畿道将举办""全家人一起参与的游戏秀""2022Play博览会京畿道是韩国游戏产业的圣地",1367,0,0,0,544332,https://weibo.com/6011831795/LtSxwgnLU?from=pa...
1,2022-05-18,京畿道杨平吉兆酒店在京畿道可以见到一个日式建筑物，具有异国风情的地方,6441,2,0,0,0,https://weibo.com/6011831795/LtAldcubF?from=pa...
2,2022-05-16,京畿道杨平大型户外咖啡店 JUN179京畿道杨平拥有广阔的空间并衔接着龙门川溪谷，是感受大自...,6199,3,2,0,0,https://weibo.com/6011831795/LthBqiDCm?from=pa...
3,2022-05-13,京畿道，举办时隔3年回归的线下游戏秀”PlayX4”📍 指南 : O网页链接#京畿道##京畿...,6723,4,0,0,0,https://weibo.com/6011831795/LsPpKv8Gi?from=pa...
4,2022-05-11,道，“创意科学培养体验中心” 开馆…培养无人机、宇宙产业等未来人才在京畿道坡州和杨平未来教育...,7074,3,0,0,0,https://weibo.com/6011831795/LswxZh38m?from=pa...


# 데이터 셋. 인덱스 내림차순 정렬

In [103]:
# 인덱스 내림차순 정렬
final_data = slicing_data.sort_index(ascending=False)
final_data.head()

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
56,2022-01-01,🎊 新年快乐! 🎊祝你2022年与京畿道一起度过幸福的一年,12213,11,0,4,0,https://weibo.com/6011831795/L8IJk9tQd?from=pa...
55,2022-01-05,京畿道支援的异色创业趋势！充分利用共享厨房的 “G-创食学院”的运营共享厨房是指多个营业者共...,14174,3,0,0,0,https://weibo.com/6011831795/L9mdF9t5G?from=pa...
54,2022-01-06,美味的冬季京畿道传统市场一到冬天就会想起鲫鱼饼、烤红薯、鱼糕、甜饼等很多零食吧？请来访问具有...,15865,5,0,0,0,https://weibo.com/6011831795/L9wcfEBPu?from=pa...
53,2022-01-07,🍅京畿道光州市代表农作物番茄的重新诞生！酸酸甜甜的番茄辣椒酱同大韩民国代表酱料辣椒酱一样受到...,16475,4,0,1,0,https://weibo.com/6011831795/L9FbKuzBl?from=pa...
52,2022-01-13,京畿道为外国人居民定居支援的事业募集📣京畿道，支援外国人居民与高丽人同胞支援事业🔥在大韩民国...,16246,5,1,1,0,https://weibo.com/6011831795/LaAyPh9ZV?from=pa...


## 엑셀 파일로 출력

In [104]:
# 파일로 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'weibo_' + file_name + '.xlsx'
path = 'crawled_data/weibo/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

weibo_20220520-174306.xlsx 파일 생성 완료


# 코드 마지막